In [1]:
from time import time
from scipy import sparse
from scipy import linalg
import matplotlib.pyplot as plt
import nibabel as nib
from sklearn.datasets import make_regression
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
import SimpleITK as sitk
import six
from radiomics import featureextractor 
from radiomics import imageoperations
import numpy as np
from numpy import linalg

In [ ]:
# Load QSM and segmentation
seg = nib.load('/media/ironman/IRON-MAN-Imaging/atlas/niis/qsm/transformed_qsm/qsm_001_20200923_2_atlas.nii.gz')
qsm = nib.load('/media/ironman/IRON-MAN-Imaging/atlas/niis/qsm/ps_seg.nii.gz')
voxel_size = qsm.header['pixdim'][0:3]

In [ ]:
seg_sitk = sitk.GetImageFromArray(seg.get_fdata())
seg_sitk.SetSpacing(voxel_size.tolist())
qsm_sitk = sitk.GetImageFromArray(qsm.get_fdata())
qsm_sitk.SetSpacing(voxel_size.tolist())
qsm_sitk_n = imageoperations.normalizeImage(qsm_sitk)

extractor0 = featureextractor.RadiomicsFeatureExtractor()
extractor0.disableAllFeatures()
feat_class = 'glcm'
extractor0.enableFeatureClassByName(feat_class)
# Perform global extraction to obtain bounding box for each ROI
Phi = []
for j in range(1,int(np.max(seg.get_fdata()))+1):
    featureVector = extractor0.execute(qsm_sitk,seg_sitk,label=j)
    Phi.append(featureVector)


In [ ]:
Phi[0]

In [67]:
x_row = []
for j in range(int(np.max(seg.get_fdata()))):
    featureVector = Phi[j]
    for key, value in six.iteritems(featureVector):
        if feat_class in key:
            x_row.append(value)

X0 = np.array(x_row)
X = sparse.coo_matrix(X0)


In [ ]:
X0

In [ ]:
plt.imshow(np.expand_dims(X0,0))
plt.xticks([])
plt.yticks([])
plt.title('Feature matrix $X$');

In [70]:
X0 = X0.T

In [ ]:
X0.shape

In [ ]:
linalg.matrix_rank(X0)

In [73]:
# UPDRS score
# Response vector u [=] n_samples x n_targets
# 1 x 4
u = np.expand_dims(np.array([0,1,24,1]).T,0)

In [74]:
# Simple 1D example
scaler = StandardScaler()
X0 = scaler.fit_transform(X0.reshape(-1,1))
X0 = X0.T

In [ ]:
X0

In [ ]:
plt.imshow(X0)
plt.xticks([])
plt.yticks([])
plt.title('Feature matrix $X$');

In [ ]:
reg = Lasso(alpha=0.1,fit_intercept=False, max_iter=10000, tol=0.1)
w_min = reg.fit(X0,u)
# reg.predict()

In [ ]:
w_min.coef_

In [81]:
# Closed form solution to L2 norm of Aw = u
L = linalg.det(np.matmul(X0,X0.T))
if abs(L) > 0:
    wc =np.matmul(X0.T,linalg.inv(np.matmul(X0,X0.T)))*np.squeeze(u)

In [ ]:
(w_min.coef_).shape

In [ ]:
X0.shape

In [84]:
# Compute u_est = Aw_min
u_est = np.matmul((X0),(w_min.coef_).T)

In [ ]:
u_est

In [86]:
mse = np.sum(np.sqrt((u-u_est)**2),1)

In [ ]:
mse

In [88]:
# Compute u_est = Aw_min
u_c = np.matmul((X0),(wc))

In [ ]:
u_c